### Question 4

min $f = x_1^2 + x_2^2 + x_3^2$

subject to $h_1 = \frac{x_1^2}{4} + \frac{x_2^2}{5} + \frac{x_3^2}{25} - 1 = 0$ and $ h_2 = x_1 + x_2 - x_3 = 0$

$\frac{\partial f}{\partial x_1} = 2 x_1$

$\frac{\partial f}{\partial x_2} = 2 x_2$

$\frac{\partial f}{\partial x_3} = 2 x_3$

$\frac{\partial h_1}{\partial x_1} = \frac{2}{4} x_1$

$\frac{\partial h_1}{\partial x_2} = \frac{2}{5} x_2$

$\frac{\partial h_1}{\partial x_3} = \frac{2}{25} x_3$

$\frac{\partial h_2}{\partial x_1} = 1$

$\frac{\partial h_2}{\partial x_2} = 1$

$\frac{\partial h_2}{\partial x_3} = -1$

total number of variables is 3. $n = 3$

2 contraints means there are 2 state variables. $m = 2$

decision variables is $n-m = 1$

so 

$\frac{\partial h}{\partial s} = 
\begin{bmatrix}
\frac{2}{5} x_2 & \frac{2}{25} x_3 \\
1 & -1 \\
\end{bmatrix}$

$\frac{\partial f}{\partial s} = 
\begin{bmatrix}
2 x_2 & 2 x_3 \\
\end{bmatrix}$

$\frac{\partial h}{\partial d} = 
\begin{bmatrix}
\frac{1}{2} x_1 \\ 1 \\
\end{bmatrix}$

$\frac{\partial f}{\partial d} = 
\begin{bmatrix}
2 x_1 \\ 
\end{bmatrix}$

coding it up:


In [156]:
import numpy as np

n = 3
m = 2
t = 0.3
lda = 1


# Partial h partial s
def phps(x): 
    phps = np.zeros([m,m])
    phps[0,0] = (2/5) * x[1]
    phps[0,1] = (2/25) * x[2]
    phps[1,0] = 1
    phps[1,1] = -1
    return phps

# Partial f partial d
def pfpd(x):
    pfpd = np.zeros([1,n-m])
    pfpd[0,0] = 2 * x[0]
    return pfpd

# Partial f partial s
def pfps(x):
    pfps = np.zeros([1,m])
    pfps[0,0] = 2 * x[1]
    pfps[0,1] = 2 * x[2]
    return pfps

# Partial h partial d
def phpd(x):
    phpd = np.zeros([m,n-m])
    phpd[0,0] = x[0] / 2
    phpd[1,0] = 1
    return phpd

# Function that is being minimized, function is useful for calculating f_alpha and phi_alpha in linesearch
def min_fun(x):
    return x[0] ** 2 + x[1] ** 2 + x[2] ** 2

# Constraints equations, used in newtons method for update state variables so it fits constrains. 
def constraints(x):
    h = np.zeros([m,1])
    h[0,0] = ((x[0] ** 2) / 4 + (x[1] ** 2) / 5 + (x[2] ** 2) / 25) - 1
    h[1,0] = x[0] + x[1] - x[2]
    return h
 
# How to calculate the reduced gradient 
def dfdd(x):
    return np.array(pfpd(x) - np.matmul(np.matmul(pfps(x),np.linalg.inv(phps(x))),phpd(x))).flatten()

# how to adjust s for 4.1 and 4.3
def s_adjustment(x):
    return np.transpose(np.matmul(np.matmul(np.linalg.inv(phps(x)),phpd(x)),np.transpose(dfdd(x)))).flatten()

# Code for 4.2 and 4.3, also used in linesearch for calculating f_alpha
def sep_x_and_adjust_and_comb(x,alpha):
    d = x.copy()[0]
    s = x.copy()[1:]
    d -= alpha * dfdd(x)
    s += alpha * s_adjustment(x)
    
    return np.concatenate((d,s))

# Linesearch code for 4.1
def linesearch(x,alpha):
    f_alpha = min_fun(sep_x_and_adjust_and_comb(x,alpha)) 
    phi_alpha = min_fun(x) - alpha*t*np.matmul(dfdd(x),np.transpose(dfdd(x)))
    
    while f_alpha > phi_alpha:
        alpha *= 0.5
        f_alpha = min_fun(sep_x_and_adjust_and_comb(x,alpha))
        phi_alpha = min_fun(x) - alpha*t*np.matmul(dfdd(x),np.transpose(dfdd(x)))
        #print(phi_alpha)
        #print(f_alpha)
        
    return alpha

# 4.4 code using newtons method to gaurentee constraint 
def solve_s(x):
    s = x[1:]
    
    j = 0
    while np.linalg.norm(constraints(x)) > tol and j < 1000:
        mat1 = phps(x)
        mat2 = np.transpose(mat1)
        mat3 = np.linalg.inv(np.matmul(mat1,mat2) + lda*np.eye(m))
        s -= np.transpose(np.matmul(np.matmul(mat3,mat1),constraints(x))).flatten()
        j += 1
    return x

def solve_problem_4(x):
    k = 0
    # main loop, does 4.1,4.2,4.3,4.4 in that order, dfdd is recalculated at several places
    while np.linalg.norm(dfdd(x)) ** 2 > tol and k < 100:
        alpha = linesearch(x,1)
        #print('4.1 x:',x,'4.1 alpha:',alpha)
        x = sep_x_and_adjust_and_comb(x,alpha)
        #print('4.2 and 4.3',x)
        x = solve_s(x)
        #print('4.4',x)

        k += 1

    print('iteration:',k)
    print(x)

    print()

# Intial condition and tolerance 
x = np.array([2*np.sqrt(5)/3, -2*np.sqrt(5)/3, 0])
tol = 1e-10

solve_problem_4(x)


iteration: 11
[ 1.5737781  -1.37705786  0.19672024]



The analytical solution to this problem, found using wolfram alpha is: 

$ \begin{pmatrix} 
20 \sqrt{\frac{2}{323}} & -\frac{35}{\sqrt{646}} & \frac{5}{\sqrt{646}}\end{pmatrix}$ or 
$ \begin{pmatrix} 
-20 \sqrt{\frac{2}{323}} & \frac{35}{\sqrt{646}} & -\frac{5}{\sqrt{646}}\end{pmatrix}$

The numerical solution is about equal to the first analytical solution. To numerically find the other, different initial conditions would need to be used. Such as multiplying the initial conditions by -1. I show this below. 

In [157]:
solve_problem_4(-x)

iteration: 11
[-1.5737781   1.37705786 -0.19672024]

